In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

In [ ]:
!pip install -qU pypdf

In [ ]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Ensure your VertexAI credentials are configured

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")



In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")


[0.04703257977962494,
 -0.04019005596637726,
 -0.02902696467936039,
 -0.026809632778167725,
 0.01892058178782463,
 -8.365760731976479e-05,
 0.03111639805138111,
 -0.019520554691553116,
 0.011491335928440094,
 0.009625785052776337,
 0.04571185261011124,
 0.051709506660699844,
 -0.007854084484279156,
 -0.07627559453248978,
 -0.000736518413759768,
 -0.022592447698116302,
 0.011496763676404953,
 -0.0076109664514660835,
 0.006400744896382093,
 -0.0036826308351010084,
 -8.639021689305082e-05,
 0.007910552434623241,
 -0.031401973217725754,
 -0.027668770402669907,
 0.013148307800292969,
 0.005762958899140358,
 -0.0022430468816310167,
 -0.07029420882463455,
 0.007011014502495527,
 0.07013051956892014,
 -0.047634560614824295,
 0.008311820216476917,
 -0.06021169573068619,
 0.01643129624426365,
 0.042709145694971085,
 -0.047674257308244705,
 0.0342608243227005,
 0.021967323496937752,
 -0.007065100595355034,
 0.00032590707996860147,
 0.013825682923197746,
 -0.08921292424201965,
 -0.0340406857430934

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "quickstart4"
pc = Pinecone(api_key='pcsk_7BsUSM_Tj1KvA2R5kyyA4F9SPU7KUu7BiDUmWYiYdWJx7aJVCSPkHRYYrDjZcsxyPqhjGh')

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("documents.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'documents.txt'}, page_content='Dr. Eleanor Harper, born on March 3, 1978, in Ashford, England, is a renowned computational linguist and artificial intelligence researcher. She grew up in a small countryside town where her fascination with languages and computers began at an early age. Eleanor was known for her curiosity, often tinkering with her father’s old typewriter and later, the family’s first computer.'), Document(metadata={'source': 'documents.txt'}, page_content='Eleanor graduated with honors in Linguistics from the University of Cambridge in 1999. She later pursued a master’s degree in Computer Science at Stanford University, where her passion for the intersection of language and technology deepened. In 2005, she earned her Ph.D. in Computational Linguistics from the Massachusetts Institute of Technology (MIT), where her groundbreaking thesis focused on machine translation using statistical methods.'), Document(metadata={'source': 'documents.txt'

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content))
    # Upsert the vector with a unique ID
    index.upsert(vectors=[(doc_id, vector)])



#


100%|██████████| 6/6 [00:01<00:00,  3.57it/s]


In [ ]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [ ]:
query = "Who was Dr. Harper?"
response = qa_chain.run(query)
print(response)

PineconeApiAttributeError: ScoredVector has no attribute 'metadata' at ['['received_data', 'matches', 0]']['metadata']